# 3. MalRNN 코드 작성

* 필요한 package 설치 확인 및 불러오기

In [9]:
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. 추출된 Stream data 에서의 시퀀스 추출 과정 구축
* 추출된 stream data(.csv)에서 필요한 byte stream data 추출

In [10]:
def parse_data(data_path, chunk_len, malconv_min_len, len_limit=100000):
    """
    :param str data_path: stream data csv파일의 위치
    :param int chunk_len: benign byte stream sampling 길이(정상데이터는 최소 chunk_len 이상이어야 함)
    :param int malconv_min_len: malconv 탐지를 위한 최소 길이(악성데이터는 최소 malconv_min_len 이상이어야 함)
    :param int len_limit: 시스템 과부하 방지 목적 길이 제한
    :return 정상/악성 byte stream data list
    """
    # TODO
    ### Byte Stream 데이터 추출을 위한 함수 만들기 ###
    # data_path에 위치한 파일 열기    
    with open(data_path, "r", encoding="cp949") as data:
        
        # 정상/악성 데이터를 저장할 list 생성
        benign_data = []
        critical_data = []

        # 파일 내 Stream data line별로 읽기 
        for line in data.readlines():
            try:
                # len_limit 이내의 데이터에서 추출
                if len(line) < len_limit:

                    # csv 내의 Stream 이후만 추출하기 위한 "]"의 위치 찾은후 이후 데이터만 추출
                    look = line.rfind("]")
                    line = line[look+2 : ]

                    # 추출된 데이터의 개행문자 제거 및 전처리
                    line = line.replace("\n", "")
                    if line.startswith(","):
                        line = line[1:]
                    line = line.split(",")

                    # 전처리된 데이터의 int 변환 및 list 삽입
                    line = [int(x,0) for x in line]
                    
                    # 정상/악성 여부에 따른 최소 길이 확인 및 list 삽입
                    if line[1] == 0 and len(line[2:]) > chunk_len:
                        ### FILL HERE ###
                    elif line[1] == 1 and len(line[2:]) > malconv_min_len:
                        ### FILL HERE ###
            except:
                pass
    
    return benign_data, critical_data

## 2. 생성된 byte stream 악성 여부 확인

In [11]:
def eval_detection(malconv, gen_bytes):
    """
    :param nn.Module malconv: malconv 모델
    :param list gen_bytes: 생성된 byte stream 
    """
    with torch.no_grad():
        gen_bytes = torch.from_numpy(np.frombuffer(gen_bytes, dtype=np.uint8)[np.newaxis, :])
        malconv_output = F.softmax(malconv(gen_bytes), dim=-1).detach().numpy()[0,1]
        return malconv_output

## 3. 정상 데이터 sampling

In [12]:
def create_benign_sample(benign_stream, chunk_len, batch_size, device):
    """
    :param list benign_stream: 정상 byte stream
    :param int chunk_len: sampling 길이
    :param int batch_size: 한번 학습에 이용할 byte 갯수
    :param device: 학습 device(CPU/GPU)에 따른 할당
    :return input_stream, target_stream
    """
    # TODO
    ### 정상 데이터 sampling 만들기 ###
    
    # input_stream(학습에 사용), target_stream(loss 산정 시 사용) 선언 (batch_size * chunk_len)
    input_stream = torch.LongTensor(batch_size, chunk_len)
    target_stream = torch.LongTensor(batch_size, chunk_len)
    
    # batch size 만큼의 for loop 생성
    for batch in range(batch_size):

        # chunk_len을 고려한 임의의 start index 선정
        start_index = ### FILL HERE ###
        
        # chunk_len을 고려한 end_index 계산
        end_index = start_index + chunk_len + 1
        
        # benign stream data에서 sampling
        chunk = ### FILL HERE ###
        
        # input_stream과 target_stream 저장
        input_stream[batch] = torch.as_tensor(chunk[:-1])
        target_stream[batch] = torch.as_tensor(chunk[1:])
    
    # 저장된 input_stream, target_stream device 할당
    input_stream = torch.LongTensor(input_stream).to(device)
    target_stream = torch.LongTensor(target_stream).to(device)
    return input_stream, target_stream   

## 4. 학습과정에 필요한 Byte Stream 생성 함수 구축

In [13]:
def generate_byte(model, base_stream, device, len_to_predict=1000, temperature=0.8):
    """
    :param nn.Module model: Byte Stream 생성할 MalRNN 모델
    :param list base_stream: 생성에 이용될 기초 byte stream
    :param device: 학습 device(CPU/GPU)에 따른 할당
    :param int len_to_predict: 생성할 Byte Stream 갯수
    :param float temperature: 분포 smoothing을 위한 지수
    """
    # TODO
    ### MalRNN을 이용해 byte stream 생성 ###

    # 입력된 base stream 적용을 위한 model의 hidden state 초기화
    hidden_state = model.init_hidden(1).to(device)

    # base stream을 학습에 적합하도록 unsqueeze를 이용한 차원 추가
    base_input = torch.LongTensor(base_stream).unsqueeze(0).to(device)
    
    # 예측 variable 선언 및 base_stream 적용
    predict = base_stream

    # 마지막 byte stream을 제외한 byte stream model hidden state에 적용
    for p in range(len(base_stream) - 1):
        _, hidden_state = model(base_input[:, p], hidden_state)

    output_result = []
    model_input = base_input[:, -1]

    # len_to_predict 길이까지 하나씩 byte stream 생성
    for p in range(len_to_predict):
        output, hidden_state = ### FILL HERE ###
        output_result.append(output)

        output_dist = output.data.view(-1).div(temperature).exp()
        predict_stream = torch.multinomial(output_dist, 1)[0]

        predict = ### FILL HERE ###
        model_input = (
            torch.tensor(predict_stream, dtype=torch.long).unsqueeze(0).to(device)
        )

    return predict.tolist(), output_result

## 5. MalRNN 학습과정 구축

* 학습에 필요한 moudle 불러오기

In [14]:
from rnn_model import CharRNN
from MalConv import MalConv

* MalRNN에 필요한 파일 경로

In [15]:
stream_data_path = "../data/3_malrnn_sample.csv"
malconv_weight_path = "../data/malconv_doc.pth"

In [26]:
def train_MalRNN():
    
    ### MalRNN 학습과정 구축 ###

    # TODO: 정상/악성 byte stream 호출
    benign_data, critical_data = ### FILL HERE ###

    # 학습장치(CPU/GPU) 호출
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print(f"Current device is {device}")

    # MalConv 로드
    malconv = MalConv(channels=256, window_size=512, embd_size=8)
    malconv_weight = torch.load(malconv_weight_path, map_location=torch.device('cpu'))
    malconv.load_state_dict(malconv_weight)

    # TODO: MalRNN 모델 구성하기 (input_size=256, hidden_size=100, output_size=256, model=gru, n_layers=1)
    model = ### FILL HERE ###

    # MalRNN 모델 학습장치 할당
    model.to(device)

    # TODO: 학습 loss function 및 optimizer 호출
    criterion = ### FILL HERE ###
    optimizer = ### FILL HERE ###

    # 학습에 필요한 variable 선언
    loss_record = []
    best_score = -1
    best_loss = -1
    best_model = None  
    
    # TODO 매 epoch별 학습 구성
    for epoch in range(1, 11):
        print(f"EPOCH {epoch}")

        # 정상 stream data sampling -> input / target benign stream data 수집
        input_benign, target_benign = create_benign_sample(
            benign_stream=### FILL HERE ###,
            chunk_len=200,
            batch_size=10,
            device=device,
        )

        # 모델 hidden state 초기화 및 학습장치 할당
        hidden_state = model.init_hidden(10)
        hidden_state.to(device)

        # 모델 parameter 초기화 및 loss variable 선언
        model.zero_grad()
        loss = 0

        # 생성을 위한 임의의 악성 stream data 추출 
        base_stream = critical_data[random.randrange(0, len(critical_data))][:1024]
        
        # 추출한 악성 stream data에 기반한 stream data 생성
        predicted, _ = ### FILL HERE ###

        # 기반 stream data와 생성된 stream data 결합
        candidate = bytearray(base_stream) + bytearray(predicted[0])
        
        # 결과물에 대한 악성 여부 malconv 확인
        malconv_result = ### FILL HERE ###
        
        # 정상 Stream data를 이용한 MalRNN 생성 학습
        for c in range(200):
            output, hidden_state = model(input_benign[:, c], hidden_state.to(device))

            # 생성한 stream과 실제 stream의 차이(loss) 계산 및 학습 반영
            loss += criterion(output.view(10, -1), target_benign[:, c])
        
        # loss 기록 및 loss를 통한 역전파 계산 및 optimizer step 이동 
        loss_record.append(loss)
        ### FILL HERE ###
        ### FILL HERE ###
        print(f"Epoch {epoch} loss: {loss.data / 200}")
        print(f"Detection Possibility: {malconv_result : 4f}")

        # loss 및 탐지 확률에 따른 모델 저장
        if epoch == 1:
            print("Saving the first model")
            best_model = model
            best_score = malconv_result
            best_loss = loss.data / 200
        elif best_score > malconv_result:
            print("Best score updated! Saving...")
            best_model = model
            best_score = malconv_result
            best_loss = loss.data / 200
        elif best_score == malconv_result:
            if best_loss > (loss.data / 200):
                print("Best score updated! Saving...")
                best_model = model
                best_score = malconv_result
                best_loss = loss.data / 200
    
    # 학습된 최종 모델 저장
    torch.save(best_model, "./malRNN_doc.pt")

In [ ]:
train_MalRNN()